In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [2]:
pth = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2'
pth_mat = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2\5x_python\cropped_tile\matfiles'

In [3]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]
slide_names = [file[:20] for file in mat_files]

# Get a list of NDPI images and their base names (without extensions)
#ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.tif')]

# Find the matching files based on their base names
matching_files = set(slide_names).intersection(ndpi_files)

# WSIs = sorted([filename + '.ndpi' for filename in matching_files])
WSIs = sorted([filename + '.tif' for filename in matching_files])

crop_mats = sorted([filename + '.mat' for filename in mat_files])

In [4]:
crop_mats

['PD_11 2O LCM.mat',
 'PD_12 1F LCM-1-1.mat',
 'PD_14 1F LCM.mat',
 'PD_15 3Q LCM.mat']

In [12]:
WSIs

['PD_11 2O LCM.tif',
 'PD_12 1F LCM-1-1.tif',
 'PD_14 1F LCM.tif',
 'PD_15 3Q LCM.tif']

In [27]:
res20x=0.4416
res2_5x=2

ratio = res2_5x/res20x*1.1


outpth = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\1024tiles\new_testing_tiles_12_5\cropped_tile\20x_tiles'
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [28]:
print(ratio)

4.981884057971015


In [29]:
crop_mats

['PD_11 2O LCM.mat',
 'PD_12 1F LCM-1-1.mat',
 'PD_14 1F LCM.mat',
 'PD_15 3Q LCM.mat']

In [30]:
WSIs

['PD_11 2O LCM.tif',
 'PD_12 1F LCM-1-1.tif',
 'PD_14 1F LCM.tif',
 'PD_15 3Q LCM.tif']

In [31]:
outpth = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\PDAC PURITY Final Analysis\LCM_H&Es_10x\czi\ometiff20xv2\5x_python\cropped_tile'

In [32]:
from scipy.io import loadmat

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,crop_mats[i][:-4]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 256
        crop_height = 256
        crop_x = int(data['crop_x']*ratio)
        crop_y = int(data['crop_y']*ratio)

        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        print(crop_x)
        print(crop_y)
        print(crop_width)
        print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Cropping PD_11 2O LCM.tif (1/4)
30484
34936
256
256
Cropping PD_12 1F LCM-1-1.tif (2/4)
28240
23851
256
256
Cropping PD_14 1F LCM.tif (3/4)
40954
21476
256
256
Cropping PD_15 3Q LCM.tif (4/4)
37481
17717
256
256


In [38]:
crop_x


19818

In [39]:
crop_y

20134